In [2]:
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
# 로이터 뉴스 데이터셋 불러오기
from tensorflow.keras.datasets import reuters
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence

numpy.random.seed(3)
tf.random.set_seed(3)

In [3]:
def loadData():
    # 불러온 데이터를 학습셋, 테스트셋으로 나누기
    (X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000, test_split=0.2)

    # 데이터 확인하기
    category = numpy.max(Y_train) + 1
    print('카테고리: {}'.format(category))
    print('학습용: {}'.format(len(X_train)))
    print('테스트용: {}'.format(len(X_test)))
    print(X_train[0])
    return (X_train, Y_train), (X_test, Y_test)

(X_train, Y_train), (X_test, Y_test) = loadData()

카테고리: 46
학습용: 8982
테스트용: 2246
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [4]:
def setXto100(X):
    X_padded = sequence.pad_sequences(X, maxlen=100)
    return X_padded

X_train = setXto100(X_train)
X_test  = setXto100(X_test)
print(X_train[0])
print(X_test[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   1   2   2   8  43
  10 447   5  25 207 270   5   2 111  16 369 186  90  67   7  89   5  19
 102   6  19 124  15  90  67  84  22 482  26   7  48   4  49   8 864  39
 209 154   6 151   6  83  11  15  22 155  11  15   7  48   9   2   2 504
   6 258   6 272  11  15  22 134  44  11  15  16   8 197   2  90  67  52
  29 209  30  32 132   6 109  15  17  12]
[  5   2   7 748  48   9   2   7 207   2 966   2 793  97 133 336   7   4
 493  98 273 104 284  25  39 338  22 905 220   2 644  59  20   6 119  61
  11  15  58 579  26  10  67   7   4 738  98  43  88 333 722  12  20   6
  19 746  35  15  10   9   2 855 129 783  21   4   2 244 364  51  16 299
 452  16 515   4  99  29   5   4 364 281  48  10   9   2  23 644  47  20
 324  27  56   2   2   5 192 510  17  12]


In [5]:
def getOneHotEncodedLabel(y):
    Y_encoded = tf.keras.utils.to_categorical(y)
    return Y_encoded

Y_train = getOneHotEncodedLabel(Y_train)
Y_test  = getOneHotEncodedLabel(Y_test)
print(Y_train[:1])
print(Y_test[:1])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
print(X_train.shape)
print(Y_train.shape)

(8982, 100)
(8982, 46)


In [7]:
def createLSTMModel():
    model = Sequential()
    model.add(Embedding(1000, 100))
    model.add(LSTM(100, activation='tanh'))
    model.add(Dense(46, activation='softmax'))
    return model

model = createLSTMModel()
model.reset_metrics()

In [8]:
def trainModel(model, x_train, y_train, x_test, y_test):
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    # 모델의 실행
    history = model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

    return model, history

trained_Model, history = trainModel(model, X_train, Y_train, X_test, Y_test )

Epoch 1/20
71/71 [==============================] - 3s 46ms/step - loss: 2.6572 - accuracy: 0.3358 - val_loss: 2.4025 - val_accuracy: 0.3620
Epoch 2/20
71/71 [==============================] - 2s 34ms/step - loss: 2.1459 - accuracy: 0.4555 - val_loss: 2.0113 - val_accuracy: 0.5036
Epoch 3/20
71/71 [==============================] - 2s 35ms/step - loss: 1.9932 - accuracy: 0.5016 - val_loss: 1.9868 - val_accuracy: 0.5098
Epoch 4/20
71/71 [==============================] - 3s 35ms/step - loss: 1.8653 - accuracy: 0.5267 - val_loss: 1.7929 - val_accuracy: 0.5503
Epoch 5/20
71/71 [==============================] - 3s 35ms/step - loss: 1.7334 - accuracy: 0.5502 - val_loss: 1.7181 - val_accuracy: 0.5552
Epoch 6/20
71/71 [==============================] - 3s 35ms/step - loss: 1.6598 - accuracy: 0.5736 - val_loss: 1.6638 - val_accuracy: 0.5824
Epoch 7/20
71/71 [==============================] - 2s 35ms/step - loss: 1.6669 - accuracy: 0.5729 - val_loss: 1.6660 - val_accuracy: 0.5793
Epoch 8/20
71

In [ ]:
def drawLossChart(history):
    # 테스트 셋의 오차
    y_vloss = history.history['val_loss']

    # 학습셋의 오차
    y_loss = history.history['loss']

    # 그래프로 표현
    x_len = numpy.arange(len(y_loss))
    plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
    plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

    # 그래프에 그리드를 주고 레이블을 표시
    plt.legend(loc='upper right')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()



In [ ]:
(X_train, Y_train), (X_test, Y_test) = loadData()
x_train = setXto100(X_train)
x_test = setXto100(X_test)
y_train= getOneHotEncodedLabel(Y_train)
y_test = getOneHotEncodedLabel(Y_test)
model = createLSTMModel()
trained_model, history = trainModel(model, x_train, y_train, x_test, y_test)

# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

drawLossChart(history)